# Import Into HDFS

In [ ]:
Source_Dir_X = "src/X_output/workfile/"
HDFS_Dir_X = "/user/cluster/AdLearn/Dev/workfile2/"
Source_Dir_Y = "src/Y_output/"
HDFS_Dir_Y ="/user/cluster/AdLearn/Dev/Y2/"

#import os
print "############################################################################"
print "##### Commandes à input dans un shell d'une machine connectée à Hadoop #####"
print "############################################################################"
print "" 

print 'if $(! hdfs dfs -test -d ' + HDFS_Dir_X + ') ; then hdfs dfs -mkdir ' + HDFS_Dir_X + '; fi'
print 'if $(! hdfs dfs -test -d ' + HDFS_Dir_Y + ') ; then hdfs dfs -mkdir ' + HDFS_Dir_Y + '; fi'

print ""
print "hdfs dfs -put -f " + Source_Dir_X +"*.csv " + HDFS_Dir_X 
print "hdfs dfs -put -f " + Source_Dir_Y +"*.csv " + HDFS_Dir_Y 

#os.system("ls")

In [ ]:
sc

# Configuration Zone

In [ ]:
CLUSTER_ADD = "hdfs://ad-cluster-master:54310"
SOURCE_DIR = "/user/cluster/AdLearn/Debug/17-05-18/"
MODEL_DIR = "/user/cluster/AdLearn/Debug/"
Y_DIR = "/user/cluster/AdLearn/Debug/17-05-18/"
PARQUET_SOURCE_FILE = "AdDF-debug12.parquet"
PARQUET_SOURCE_DIR = "/user/cluster/AdLearn/Debug/"
Y_FILE = "EW_YF3MCN.csv"

num_cores = 18
gran = 2
Force_regen = False

BaseNameMergedFile = "merged_forHDFS2"
Const_HEADER = True

In [ ]:
import gc
import time
import sys
import random
import math as m
import functools
from itertools import islice

import numpy as np
import pandas as pd
%matplotlib inline
import matplotlib.pyplot as plt
import seaborn as sns

from pyspark.mllib.linalg import Vector, Vectors
from pyspark.sql.functions import col, count, sum
from pyspark.mllib.stat import Statistics
from pyspark.mllib.linalg.distributed import *

from hdfs import Config

# Aggregation into HDFS

In [ ]:
def multipleJoins(x,y):
    if isinstance(x, list) and isinstance(y, list):
        return sorted(x + y)
    elif isinstance(x, list):
        return sorted(x + [y])
    elif isinstance(y, list):
        return sorted([x] + y)
    else:
        return sorted([x,y])

try:
    Const_HEADER
except NameError:
    sys.exit("Loading Configuration is needed !")
    
client = Config().get_client()
Parquet_List = client.list(PARQUET_SOURCE_DIR)
regen = False if (PARQUET_SOURCE_FILE in Parquet_List) else True


if regen == True or Force_regen == True :
    t1 = time.time()
    print "##########################"
    print "#### TASK: Files gathering"
    print "Model Loading"
    rdd = sc.textFile(CLUSTER_ADD + MODEL_DIR + 'model.csv')
    rddDate = rdd.zipWithIndex().filter(lambda ind: ind[1] > 0).map(lambda (x,y): (y,x)).map(lambda x: (x[0],x[1].split(",")[0]))
    rddAction = rdd.zipWithIndex().filter(lambda ind: ind[1] > 0).map(lambda (x,y): (y,x)).map(lambda x: (x[0],x[1].split(",")[1]))

    I = [rddDate, rddAction]
    header = ["Date", "Ids"]

    n = rddDate.count() + 1

    print "X vars Loading"
    rddFile = sc.textFile(CLUSTER_ADD + SOURCE_DIR + BaseNameMergedFile + '_X.csv', (gran+1) * num_cores)
    rddX_ = rddFile.map(lambda x: np.nan if x=='""' else x).zipWithIndex().map(lambda x: (x[1] % n , x[0]))
    rddX  = rddX_.filter(lambda ind: ind[0] > 0).map(lambda (x,y): (x,float(y)))

    header_ = sc.textFile(CLUSTER_ADD + SOURCE_DIR + BaseNameMergedFile + '_X_header.csv').collect()

    header = header + header_

    I.append(rddX)

    print "Y vars Loading"
    rddFile = sc.textFile(CLUSTER_ADD + Y_DIR + BaseNameMergedFile +'_Y.csv')

    rddY_ = rddFile.map(lambda x: np.nan if x=='""' else x).zipWithIndex().map(lambda x: (x[1] % n , x[0]))
    rddY  = rddY_.filter(lambda ind: ind[0] > 0).map(lambda (x,y): (x,float(y)))

    header_ = sc.textFile(CLUSTER_ADD + Y_DIR  + BaseNameMergedFile + '_Y_header.csv').collect()

    header = header + header_

    I.append(rddY)
    
    print("#############################")
    print("TASK: RDD to DF:")
    
    uni2 =  sc.union(I).zipWithIndex().map(lambda ind: [ind[0][0],(ind[1], ind[0][1])]).coalesce((gran) * num_cores)

    s = uni2.reduceByKey(multipleJoins).map(lambda x: x[1]).map(lambda tuples: [x[1] for x in tuples])

    df = s.toDF(header)
    
    print("#############################")
    print("TASK: Save Aggregated to HDFS:")
    df.write.parquet( CLUSTER_ADD + PARQUET_SOURCE_DIR + PARQUET_SOURCE_FILE)

print "FILE AGGREGATION FINISHED"
t2 = time.time()
print("####################################")
print("TIME TAKEN FOR TASK:" + str(t2-t1))
#check_integrity(df,FilesList)

# Integrity Test (non retested for new Aggregation)

In [ ]:
#check_integrity
client = Config().get_client()
FilesList = client.list(SOURCE_DIR)
header = df2.columns
seed = time.time()
random.seed(seed)
num_check = int(len(FilesList) * 1 / 10)

print(num_check)

for i in xrange(len(FilesList)):
    id = random.randint(0,len(FilesList) - 1)
    print("WORK: " + str(i) + " ; " + str(id))
    
    with client.read(SOURCE_DIR + FilesList[i]) as reader:
        dfPandas = pd.read_csv(reader)
    
    dfCheck = df2.orderBy("Date", "Ids").select([header[2+i]]).toPandas()
    assert dfPandas.equals(dfCheck), "Integrity Check Failure: " + str(header[2+i]) + \
                                                "id: " + str(2+i) +" seed: " + str(seed)

# Loading Data from HDFS

In [ ]:
print("LOADING PARQUET FILE DATAFRAME")
t1 = time.time()
df = spark.read.parquet( CLUSTER_ADD + PARQUET_SOURCE_DIR + PARQUET_SOURCE_FILE)
t2 = time.time()
print("####################################")
print("TIME TAKEN FOR TASK:" + str(t2-t1))

print("#############################")
print("Parquet loading TASK FINISHED")
print("\n")
    
print("Dataframe LOADED")

#dfPersist = df.persist(StorageLevel.MEMORY_AND_DISK)

# Sampling Data

In [ ]:
ChosenVar = df.columns.index("EWYF3MCN")

h_ = df.columns[2:-4]
h_.append(df.columns[ChosenVar])
h_

dateDebut = df.select("Date",h_[-1]).orderBy("Date").dropna().select("Date").limit(1).rdd.map(lambda x: list(x)[0]).take(1)[0]


dfSample = df.filter(col("Date") >= dateDebut).filter(col("Date") < '2016-01-01')\
                .orderBy("Date","Ids").select(h_).persist()
    
dY = dfSample.select(h_[-1])

dY.show()
dfSample.count()

In [ ]:
df.select("Date").orderBy("Date", ascending = False).show()

In [ ]:
def isPos(x):
    if x > 0:
        return 1
    if x < 0:
        return -1
    return np.nan


from pyspark.sql.functions import udf
from pyspark.sql.types import *

udf_discretize = udf(isPos, IntegerType())
dY_log = dY.withColumn("Y_log", udf_discretize(h_[-1])).select("Y_log")

# Compute ExpY and MeanY

In [ ]:
t1 = time.time()
t = dY.filter(dY[dfSample.columns[-1]] != np.nan).rdd.map(lambda data: Vectors.dense([float(c) for c in data]))
b = Statistics.colStats(t)
Ecart_Y = m.sqrt(b.variance())
Esp_Y = b.mean()[0]
t2 = time.time()

print(t2-t1)
print(Ecart_Y)
print(Esp_Y)

In [ ]:
t1 = time.time()
t = dY_log.filter(dY_log["Y_log"] != np.nan).rdd.map(lambda data: Vectors.dense([float(c) for c in data]))
b = Statistics.colStats(t)
Ecart_Y = m.sqrt(b.variance())
Esp_Y = b.mean()[0]
t2 = time.time()

print(t2-t1)
print(Ecart_Y)
print(Esp_Y)

In [ ]:
dfSample = dfSample.withColumn("Y_log", udf_discretize(h_[-1])).select(h_[:-1] + ["Y_log"]).persist()

# Distributed Toolbox

In [ ]:
# take 'rdd' of n * p size with n - 1 features vector and 1 y vector
# take 2 extrema for comparison
# Takes out y vector
# Return activation vector by comparing with the extrema
def map_booleanRuling(x, min_b, max_b):
    row = np.array(x[:-1])
    bool_ = np.logical_and(min_b <= row, row <= max_b).astype(int)
    
    return np.append(bool_, x[-1])

# Vectoriel sum
# "Vertical sum" in terms of rdd
# Last step of "internal" Dot Product
def red_vectSum(x,y):   
    return np.array(x) + np.array(y)

def red_vectSum_withoutLast(x,y):
    return red_vectSum(x[:-1],y[:-1])
             
# Take rdd 'x' and boolean selector of size n - 1.
# x is of size n * p is n - 1 features vectors + 1 y vector (last column)
# return the m selected vectors of 'x' and y vector in one rdd
def map_selectActiv(x,boolCol):
    y = np.array(x[:-1])[boolCol]
    
    return np.append(y,x[-1]).tolist()

# take rdd 'x' of size n * p with n - 1 features vectors and 1 y vector
# return rdd with each x_i,j element multiplied by associated y_i 
# First step of "internal" Dot Product
# Erase y_i column in returned rdd
def map_Internal_YProduct(x):

    return (np.array(x[:-1]) * x[-1])

# FACADE function (design pattern)
# wrapping up all error function adapted for the current learning
def computeError(rdd, esp_cond, nb_act, function = "MSE"):
    if function == "MSE":
        
        return computeMSE(rdd, esp_cond, nb_act)
    else:
        
        raise Exception("No valid error function provided !")

        
def map_MSE(x, esp):
    exp = (esp - x[-1]) ** 2
    x = np.array(x[:-1])
    return np.multiply(x, exp) 
        
    
# MSE loss function
def computeMSE(rdd, esp_cond, nb_act):
    esp_cond = np.array(esp_cond)
    boolTest = esp_cond >= 0
    esp_cond = np.where(boolTest, 1, -1)
    rdd = rdd.map(functools.partial(map_MSE, esp=esp_cond))
    
    rdd = rdd.reduce(red_vectSum)
    array = np.true_divide(np.array(rdd), nb_act)
    
    return array

def computeCondProb(rdd, nb_act):
    #Map = Matrix of (x_i,j * y_i)
    #Reduce = Vertical Sum
    # Map + Reduce = X . Y_transpose
    dotProduct = rdd.map(map_Internal_YProduct).reduce(red_vectSum)
    
    # X.Y / nb_activation sur les règles qui passent le filtre de coverage
    return np.true_divide(np.array(dotProduct), nb_act)
    
def computeZscore(prob_cond, nb_act, esp_y, ecart_y):
    deltaCond = np.absolute(prob_cond - esp_y)
    
    # | Esp_Cond_Y - Esp_Y | / (Ecart_Y / Racine(nb_activation)) 
    Zscore = np.multiply(deltaCond, np.sqrt(nb_act) / ecart_y)
    return Zscore
        
def red_vectSum_withoutY(x,y):
    return red_vectSum([x[0]], [y[0]])

# take 'rdd' of n * p size with n - 1 features vector and 1 y vector
# take 2 extrema for comparison
# Takes out y vector
# Return activation vector by comparing with the extrema
def map_booleanRuling_list(x, min_b, max_b):
    row = np.array(x[:-1])
    bool_test = np.logical_and(row >= np.array(min_b), row <= np.array(max_b))
    
    return np.append(np.where(bool_test,1,0), x[-1])

In [ ]:
B = dfSample.na.replace(np.nan, 'NA').agg(*[
    count(c).alias(c)    # vertical operations in SQL ignore NULL
    for c in dfSample.columns
])
temp = np.array(B.rdd.collect())
h = temp[0]
totalRDDrows = np.array(h[:-1])

In [ ]:
totalRDDrows

In [ ]:
try :
    sourceDF
except NameError:
    pass
else:
    "SourceDF already assigned, unpersist to force GC on dependencies "
    sourceDF.unpersist()
    
h_ = dfSample.columns
sourceDF = dfSample.fillna(-1, subset=h_[:-1]).dropna(subset=h_[-1])
sourceDF = sourceDF.persist(StorageLevel.MEMORY_AND_DISK)

sourceDF_cover = dfSample.fillna(-1, subset=h_[:-1]).fillna(0, subset=h_[-1])
sourceDF_cover = sourceDF.persist(StorageLevel.MEMORY_AND_DISK)


h_ = sourceDF_cover.columns  